# User inferer

This examples shows how user-inferer can be implemented to agument a new attribute to existing brails-created inventory. This feature is useful when user know the rulesets to infer a new attribute (e.g. contents value) from existing attributes (e.g. occupancy type and building replacement cost).

## Formatting the user-defined inferer file

Let's first set the path to user-inferer python file. We will use "content_value_inferer.py" to infer contents value using occupancy type and building replacement cost information provided by NSI.

In [ ]:
import os
cwd = os.getcwd()
filepath =  os.path.join(cwd,"content_value_inferer.py")

The user-inferer python file should contain a function named **user_inferer**. Let's first see how the function looks like. The input **inventory_dict** and outputs **new_features** are dictionaries and their examples will be given subsequently.

In [ ]:
# Read the file contents
with open(filepath, 'r') as f:
    file_contents = f.read()

# Display the content with Markdown
from IPython.display import display, Markdown
display(Markdown(f'The contents of {filepath}'))
display(Markdown(f'---'))
display(Markdown(f'```python\n{file_contents}\n```'))
display(Markdown(f'---'))


It enumerates the existing inventory in **inventory_dict** and creates **new_feature** dictionary that contains the new attributes

An example of **inventory_dict** provided by Brails would look like below.

---
```json
inventory_json = {
    0: {
        "type": "Building",
        "properties": {
            "type": "Building",
            "buildingheight": "NA",
            "erabuilt": 1983,
            "numstories": 1,
            "roofshape": "flat",
            "fpAreas": 27433,
            "lon": -81.92019722,
            "lat": 26.43725715,
            "fparea": 32663.7,
            "repaircost": 3968655.62,
            "constype": "W1",
            "occupancy": "COM1",
            "fd_id": 497575843
        },
        "geometry": {
            "type": "Polygon",
            "coordinates": [
                [-81.9202572, 26.4375827],
                [-81.920495, 26.4370076],
                [-81.9201985, 26.4369093],
                [-81.9201437, 26.4368912],
                [-81.919906, 26.4374663],
                [-81.9202572, 26.4375827]
            ]
        }
    },
    1: {
        "type": "Building",
        "properties": {
            "type": "Building",
            "buildingheight": "NA",
            "erabuilt": 1983.0,
            "numstories": 1.0,
            "roofshape": "flat",
            "fpAreas": 8238,
            "fparea": 605.35504,
            "repaircost": 212759.348,
            "constype": "W1",
            "occupancy": "RES1"
        },
        "geometry": {
            "type": "Polygon",
            "coordinates": [
                [-81.9191106, 26.438107],
                [-81.9190345, 26.4381961],
                [-81.9189942, 26.4382432],
                [-81.9189849, 26.4382368],
                [-81.9189378, 26.4382919],
                [-81.9188165, 26.4382088],
                [-81.9188396, 26.4381817],
                [-81.9187882, 26.4381466],
                [-81.9188034, 26.4381288],
                [-81.9187612, 26.4380998],
                [-81.9187527, 26.4380479],
                [-81.918853, 26.4379305],
                [-81.9191106, 26.438107]
            ]
        }
    },
    .....
}
```
---

Note that it takes the building id as key and contains the existing attributes under the key "properties". 

The resulting **new_features** would look like.

---
```json
new_features = {
    0: {
        "contentsValue": 3968655.62
    },
    1: {
        "contentsValue": 106379.674
    },
    .....
}
```
---
    

## Generation of base inventory

Before running user-inferer, let's create a baseline inventory using NSI attributes, OSM footprint info, and imputation.

### Scraping OSM

In [ ]:
import sys
import copy

sys.path.insert(0, "../../")
from brails.utils import Importer
from brails.types.image_set import ImageSet    
from brails.types.asset_inventory import Asset, AssetInventory
importer = Importer()

In [ ]:
region_data = {"type": "locationName", "data": "Fort Myers Beach"}
region_boundary_class = importer.get_class("RegionBoundary")
region_boundary_object = region_boundary_class(region_data)

In [ ]:
#
# Get Footprints using OSM
#

print("Trying OSM_FootprintsScraper ...")

osm_class = importer.get_class("OSM_FootprintScraper")
osm_data = {"length": "ft"}
osm = osm_class(osm_data)
osm_inventory = osm.get_footprints(region_boundary_object)


### Scraping NSI atributes and merging them with OSM

In [ ]:
nsi_class = importer.get_class("NSI_Parser")
nsi = nsi_class()

In [ ]:
my_inventory = nsi.get_filtered_data_given_inventory(osm_inventory, "ft")

In [ ]:
# There can be missing attributes
my_inventory.get_asset_features(1)

### Imputing missing attributes

In [ ]:
knn_imputer_class = importer.get_class("KnnImputer")
imputer=knn_imputer_class(my_inventory,n_possible_worlds=10, exclude_features=["lon","lat","fd_id"])
fort_myers_imputed = imputer.impute()

In [ ]:
fort_myers_imputed.get_asset_features(1)

Now the base inveotires of Fort Myers Beach is created. The filtered NSI inventory does not contain information on "contentsValue". We want to add this through the user-inferer

# Example 1: Run user-inferer to update the content values

In [ ]:
user_inferer_class = importer.get_class("UserInferer")
inferer=user_inferer_class(fort_myers_imputed,filepath)
fort_myers_inferred = inferer.infer()

In [ ]:
fort_myers_inferred.get_asset_features(55)[1]

The contentsValues are now added. Note that, because there are multiple possible worlds of occupancy types (coming from probablistic imputation), contents value can be  evaluated differently for each world

# Example 2: Run-user inferer to update the floor area

Let us import another user-inferer script to estimate the floor area of average and maximum plan area ('fpAreas' and 'fpAreas_max') using occupancy type information.

Note that 'fpAreas' already exist in your inventory. You can either overwrite the existing one or not. By default, it overwrites the existing values. 

In [ ]:
import os
cwd = os.getcwd()
filepath_fp =  os.path.join(cwd,"floor_area_inferer.py")

In [ ]:
inferer=user_inferer_class(fort_myers_imputed,filepath_fp)
fort_myers_inferred_fp = inferer.infer()

In [ ]:
fort_myers_inferred_fp.get_asset_features(1)[1]

### You can also avoid the overwritting of already existing values

In [ ]:
inferer=user_inferer_class(fort_myers_imputed,filepath_fp, overwrite=False)
fort_myers_inferred_fp2 = inferer.infer()

In [ ]:
fort_myers_inferred_fp2.get_asset_features(1)[1]